In [ ]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
#import nltk

In [ ]:
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')
ftd.dtype.names = ('img_id', 'description')

In [ ]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [ ]:
all_x = mk_list(fe['pre_que'])


In [ ]:
##check blanks
## As there are some rows where there is only one image or blank/erronous question. We check that and save those indices.
## create blank list

bl =[]

for img,q in zip(enumerate(all_x), mk_list(qa['que'])):
    if (len(img[1]) <2 or len(q) < 2) and img[0] not in bl:
        bl.append(img[0])


In [ ]:
## Now for each row in the list above, find description and add it
all_i_txt1 = []
original_ind = []
bl_ind = []

for rown in range(len(all_x)):
    if rown in bl:
        bl_ind.append(rown)
    else:
        #print(all_x.index)
        int_i_txt = []
        for img in all_x[rown]:
            if img in ftd['img_id']:
                d = str(ftd['description'][list(ftd['img_id']).index(img)])
                int_i_txt.append(str(d))
        original_ind.append(rown)
        all_i_txt1.append(int_i_txt)
        

In [ ]:
## add question and answer to each data point
all_i_txt = [row + [qa['que'][rn].strip(), qa['ans'][rn].strip()] for rn, row in zip(original_ind, all_i_txt1)]


In [ ]:
## Segregate indices where answers are yes or No
yes_ind = []
no_ind = []
all_i_txt_no  = []
all_i_txt_yes = []
for i in range(len(all_i_txt)):
    if all_i_txt[i][-1] == 'No':
        all_i_txt_no.append(all_i_txt[i])
        no_ind.append(i)
    else:
        all_i_txt_yes.append(all_i_txt[i])
        yes_ind.append(i)

In [ ]:
#print(all_i_txt[0])
##basic model
tf = TfidfVectorizer()

In [ ]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = tf.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [ ]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])


In [ ]:
## Sort all the above similarities with argsort which gives indices
## Sort in descending order if the answer is yes else ascending order

sorted_ind = []

for si, v in zip(all_sim_q, [i[-1] for i in all_i_txt]):
    if v == "Yes":
        y_top = si.argsort()[::-1]
        #print(y_top)
    else:
        y_top = si.argsort()
        #print(y_top)
    sorted_ind.append(y_top)

In [ ]:
## with above indices, create new sorted array per que, this is done to then find max step difference 
sorted_sim_all = []
for q,w in zip(sorted_ind, all_sim_q):
    sorted_sim = []
    for e in q:
        sorted_sim.append(w[e])
    sorted_sim_all.append(sorted_sim)


In [ ]:
##find the difference between two similarity score in each array.
sim_dif_all = []

for d in range(len(sorted_sim_all)):
    sim_dif = [sorted_sim_all[d][f-1] - sorted_sim_all[d][f] for f in range(1, len(sorted_sim_all[d]))]
    sim_dif_all.append(sim_dif)
        


In [ ]:
## Find maximum difference in similarity scores per row and also find the indice where to cut-off

max_dif = []
max_dif_ind =[]
for g in range(len(sim_dif_all)):
    
    maxi= max(sim_dif_all[g])
    max_dif.append(maxi)
    max_dif_ind.append(sim_dif_all[g].index(maxi))


In [ ]:
## Cut off sorted indices till maximum difference index

sorted_ind_lim = [indi[0:max_indi+1] for indi,max_indi in zip(sorted_ind, max_dif_ind)]

In [ ]:
##clean_all_x- Without blanks
all_x_clean = [all_x[i] for i in original_ind]

In [ ]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred

In [ ]:
## images predicted to be eliminated

pred_img = mk_imgid(zip(all_x_clean, sorted_ind_lim))

In [ ]:
all_cheat1 = mk_list(fe['curr_elim'])
all_cheat = [all_cheat1[i] for i in original_ind]

In [ ]:
pre_all= [len(set(a).intersection(set(p)))/len(set(p)) if len(p) > 0 else 0 for a,p in zip(all_cheat, pred_img)]
rec_all = [len(set(a).intersection(set(p)))/len(set(a)) if len(a) > 0 else 0 for a,p in zip(all_cheat, pred_img)]



In [ ]:
fs_all= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all, rec_all)]

In [ ]:
pre_mean = np.mean(np.array(pre_all))
rec_mean = np.mean(np.array(rec_all))
fs_mean = np.mean(np.array(fs_all))

In [ ]:
op = ["22", "CV", "default", "Cosinesimilarity, ranking difference and answers",str(pre_mean), str(rec_mean), str(fs_mean)]

In [ ]:
with open('Results with description.txt', 'a') as f:
    f.write('; '.join(op)+ '\n')